In [2]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests
import re
import pickle

### Movie name & Consensus

In [2]:
def name_consensus_fn(soup):
    try:
        movie_name = soup.find('div',{'class':'mop-ratings-wrap score_panel js-mop-ratings-wrap'})
        movie_name = soup.findAll('h1',{'class':'mop-ratings-wrap__title mop-ratings-wrap__title--top'})[0].get_text()
    except:
        print("something wrong with movie name")
        movie_name = np.NaN
        
#     critic_consensus = soup.findAll('p',{'class':'mop-ratings-wrap__text mop-ratings-wrap__text--concensus'})[0].get_text()

    try:
        critic_consensus = soup.findAll('p',{'class':'mop-ratings-wrap__text mop-ratings-wrap__text--concensus'})[0].get_text()
    except:
        print("something wrong with critic_consensus")
        critic_consensus = np.NaN
    return movie_name, critic_consensus

### Ratings

In [3]:
def critic_score_fn(critic):
    try:
        critic_score = critic.find('span',{'class':'mop-ratings-wrap__percentage'}).get_text()
    except AttributeError:
        critic_score = np.NaN
#     print(critic_score)
    return critic_score

In [4]:
def num_critics_fn(critic):
    try:
        num_critics = critic.find('small',{'class':'mop-ratings-wrap__text--small'}).get_text()
    except:
        print("something wrong with num_critics")
        num_critics = np.NaN
#     print(num_critics)
    return num_critics

In [5]:
def audience_score_fn(audience):
    try:
        audience_score = audience.find('span',{'class':'mop-ratings-wrap__percentage'}).get_text()
    except AttributeError:
        audience_score = np.NaN
#     print(audience_score)        
    return audience_score

In [6]:
def num_audience_fn(audience):
    try:
        num_audience = audience.find('strong',{'class':'mop-ratings-wrap__text--small'}).get_text()
    except:
        print("something wrong with audience")
        num_audience = np.NaN
#     print(num_audience)
    return num_audience

### Movie info

In [7]:
def synopsis_fn(tables):
    try:
        synopsis = tables.findAll('div',{'class':'movie_synopsis clamp clamp-6 js-clamp'})[0].get_text()
    except:
        synopsis = np.NaN
#     print(synopsis)
    return synopsis

In [8]:
def movie_info_fn(movie_info_box):
    dict_data = {}

    for i in range(12):
        try:
            ii = movie_info_box[i].findAll('div',{'class':'meta-label subtle'})[0].get_text()
            dict_data[ii] = movie_info_box[i].find('div',{'class':'meta-value'}).get_text()
    #         print(synopsis[i].findAll('div',{'class':'meta-label subtle'})[0].get_text(), synopsis[i].find('div',{'class':'meta-value'}).get_text())
        except:
            continue
    return dict_data

def pg_fn(tables):
    pg = tables.findAll('div',{'class':'meta-value'})[0].get_text()
    return pg

def genre_fn(small_box):
    genre = small_box[1].find('div',{'class':'meta-value'}).get_text()
    return genre    

def directors_fn(small_box):
    directors = small_box[2].findAll('div',{'class':'meta-value'})[0].get_text()
    return directors

def written_by_fn(small_box):
    written_by = small_box[3].findAll('div',{'class':'meta-value'})[0].get_text()
    return written_by

def release_date_fn(small_box):
    release_date = small_box[4].findAll('div',{'class':'meta-value'})[0].get_text()
    return release_date

def run_time_fn(small_box):
    run_time = small_box[7].findAll('div',{'class':'meta-value'})[0].get_text()

def studio_fn(small_box):
    try:
        studio = small_box[8].findAll('div',{'class':'meta-value'})[0].get_text()
    except IndexError:
        studio = np.NaN
    return studio

### Cast

In [9]:
def list_celeb_names_fn(all_celebrities):
    top_5_celeb_names = []
    try:
        for i in range(5):
            top_5_celeb_names.append(all_celebrities[i].attrs['href'][11:])
    except IndexError:
        pass
    return top_5_celeb_names

In [10]:
def scrape_celeb_names_fn(soup):
    cast = soup.find('div',{'class':'castSection'})
    top_5_celeb_names = []
    try :
        all_celebs = cast.findAll('a',{'class':'pull-left'})
        top_5_celeb_names = list_celeb_names_fn(all_celebs)
        top_5_celeb_names = ", ".join(top_5_celeb_names)
    except AttributeError:
        pass
    return top_5_celeb_names

### Critics review

In [11]:
def critic_line_url_sub_fn(critic_lines):
    url =[]
    critic_line =[]
    for i in range(20):
        try:
            url.append(critic_lines[i].attrs['cite'])
            critic_line.append(critic_lines[i].get_text())
        except:
            continue
    return critic_line, url

In [12]:
def critic_line_url_fn(soup):
    try:
        critic_lines = soup.findAll('div',{'class':'panel-body content_body'})[3]
        critic_lines = critic_lines.findAll('blockquote',{'class':'media-body quote_bubble__quote'})
        critic_line, critic_url = critic_line_url_sub_fn(critic_lines)
        critic_line, critic_url = ", ".join(critic_line), ", ".join(critic_url)
    except:
        critic_line = np.NaN
        critic_url = np.NaN
    return critic_line, critic_url

### Audience review

In [13]:
def audience_line_sub_fn(audience_review):
    audience_line =[]
    for i in range(10):
        try:
            audience_line.append(audience_review.findAll('div',{'class':'mop-audience-reviews__review--comment clamp clamp-4 js-clamp'})[i].get_text())
        except:
            continue
    return audience_line

In [14]:
def audience_review_fn(soup):
    audience_review = soup.find('ul',{'class':'mop-audience-reviews__reviews-wrap clearfix'})
    audience_lines = audience_line_sub_fn(audience_review)
    audience_review = "".join(audience_lines)
    return(audience_review)

### Quotes 

In [15]:
def quotes_helper_fn(quotes_lines):
    quote_lines = quotes_lines[1::2]
    cleaned_quotes = []
    for line in quote_lines:
        try:
            line = str(line).replace('\n','')
            cleaned_quotes.append(re.search('<span>(.*)</span>',line).group(1))
        except:
            continue
    quotes = "".join(cleaned_quotes)
    return quotes

In [16]:
def quotes_fn(soup):
#     quotes_lines = [
    try:
        quotes_url = soup.findAll('div',{'class':'panel-body content_body'})[4]
        quotes_url = quotes_url.find('a',{'class':'unstyled articleLink fr'}).attrs['href']
        quotes_url = 'https://www.rottentomatoes.com' + quotes_url
        quotes_page = requests.get(quotes_url)
        quotes_page=bs(quotes_page.text)
        quotes_lines = quotes_page.findAll('div',{'class':'panel-body content_body'})[1]
        quotes_lines = quotes_lines.findAll('span')
        
    except:
        return np.NaN
    
    return quotes_helper_fn(quotes_lines)

## 27 K movies list

In [17]:
# import pandas as pd
# import glob

# print(glob.glob('*.csv'))

# final_df = pd.DataFrame()
# list_moovies = ['dataminer_0 (2).csv',
#  'dataminer_10000 (2).csv',
#  'dataminer_20000 (1).csv',
#  'dataminer_30000 (1).csv',
#  'dataminer_40000.csv',
#  'dataminer_50000.csv']

# for i in list_moovies:
#     df = pd.read_csv(i)
#     df.columns = ['url','name']
#     print(df.tail(3))
#     final_df = final_df.append(df)
    
# final_df = final_df.dropna()
# final_df = final_df.iloc[308:]
# final_df = final_df[:-22]
# final_df.to_csv("27K_movies_url.csv",index=False)

### Scraping to get movie names

In [18]:
# page = requests.get("https://www.rottentomatoes.com/top/bestofrt/top_100_action__adventure_movies/")
# soup=bs(page.text)

# all_genre_100 = soup.findAll('ul',{'class':'dropdown-menu'})
# little_cleaned_genre_100 = all_genre_100[0].findAll('a')

# links_100_movies_various_genre = []

# for i in range(17):
#     extension = str(little_cleaned_genre_100[i].attrs['href'])
#     extension = "https://www.rottentomatoes.com" + extension
#     links_100_movies_various_genre.append(extension)

# movie_url_genre = pd.DataFrame(columns=['url'])

# for i in links_100_movies_various_genre:
#     try:
# #         print(i)
#         movie_links_genre = pd.DataFrame()
#         page = requests.get(i)
#         soup=bs(page.text)

#         movie_genre = soup.find('button',{'class':'dropdown-toggle btn btn-primary-border fullWidth'}).get_text()
#         movie_genre = str(movie_genre).replace('\n','')
#         movie_genre = str(movie_genre).replace(' ','')
#         full_table = soup.find('table',{'class':'table'})
#     #     print(full_table)
#         movies_list_url = []
#         for i in range(100):
#             url_uncleaned = full_table.findAll('a',{'class':'unstyled articleLink'})[i].attrs['href']
#             url_cleaned = 'https://www.rottentomatoes.com' + url_uncleaned
#             movies_list_url.append(url_cleaned)

#     #     print(movies_list_url)

#         movie_links_genre = pd.DataFrame(movies_list_url, columns = ['url'])
#         movie_links_genre['genre'] =  movie_genre
#         movie_url_genre = movie_url_genre.append(movie_links_genre)  
#     except:
#         continue
        
# movie_url_genre.to_csv("movie_url_genre.csv", index=False)

In [3]:
movie_name_url_27k = pd.read_csv("27K_movies_url.csv")
movie_name_url_27k = movie_name_url_27k[~movie_name_url_27k.url.str.contains("/m/null")]
movie_name_url_27k = movie_name_url_27k[movie_name_url_27k.url.str.contains("/m")]
movie_name_url_27k = movie_name_url_27k[~movie_name_url_27k.url.str.contains("editorial")]
movie_name_url_27k['url'] = 'https://www.rottentomatoes.com' + movie_name_url_27k.url
movie_name_url_27k.head()

,url,name
0,https://www.rottentomatoes.com/m/mulan_2020,Mulan 76%Available Sep 4
1,https://www.rottentomatoes.com/m/bill_and_ted_...,Bill & Ted Face the Music 81%Available Aug 28
2,https://www.rottentomatoes.com/m/i_hate_new_york,I Hate New York 100%Available Sep 1
3,https://www.rottentomatoes.com/m/get_duked,Get Duked! 86%Available Aug 28
4,https://www.rottentomatoes.com/m/the_garden_le...,The Garden Left Behind 92%Available Sep 8


In [4]:
movie_name_url_27k.shape

(26834, 2)

In [6]:
# movie_url_genre = pd.read_csv("movie_url_genre.csv")

In [25]:
def scrape_rotten(movies_list):
    final_df = pd.DataFrame()
    i=0
    for x in movies_list.url:
        try:

            print(x)
            print(i)
            i+=1
            page = requests.get(x)
            soup=bs(page.text)

            movie_name, critic_consensus = name_consensus_fn(soup)

            critic = soup.find('div',{'class':'mop-ratings-wrap__half'})
            critic_score = critic_score_fn(critic)
            num_critics = num_critics_fn(critic)

            audience = soup.find('div',{'class':'mop-ratings-wrap__half audience-score'})
            audience_score = audience_score_fn(audience)
            num_audience = num_audience_fn(audience)

            tables = soup.find('div',{'class':'panel-body content_body'})
            synopsis = synopsis_fn(tables)
    #         pg = pg_fn(tables)

            movie_info = tables.findAll('li',{'class':'meta-row clearfix'})
            movie_info_dict = movie_info_fn(movie_info)
    #         genre = genre_fn(small_box)
    #         directors = directors_fn(small_box)
    #         written_by = written_by_fn(small_box)
    #         release_date = release_date_fn(small_box)
    #         run_time = run_time_fn(small_box)
    #         print(run_time)
    #         studio = studio_fn(small_box)


            top_5_celeb_names = scrape_celeb_names_fn(soup)
            critic_line, critic_url = critic_line_url_fn(soup)
            audience_review = audience_review_fn(soup)
            quotes = quotes_fn(soup)


    #         print("=="*30)

            df=pd.DataFrame([movie_info_dict])
            df['movie_name'] = [movie_name]
            df['critic_consensus'] = critic_consensus

            df['critic_score'] = [critic_score]
            df['audience_score'] = [audience_score]
    #         df['pg'] = pg
    #         df['genre'] = genre
    #         df['directors'] = directors
    #         df['written_by'] = written_by
    #         df['release_date'] = release_date
    #         df['run_time'] = run_time
    #         df['studio'] = studio
            try:
                df['top_5_celeb_names'] = top_5_celeb_names
            except ValueError:
                df['top_5_celeb_names'] = np.NaN
            df['critic_line'] = critic_line
            df['critic_url'] = critic_url
            df['audience_review'] = audience_review
            df['quotes'] = quotes

            final_df = final_df.append(df)
        except:
            print(i)
            final_df.to_csv("27k_data.csv")
    return final_df

In [26]:
movies_27k_info = scrape_rotten(movie_name_url_27k)
movies_27k_info = movies_27k_info.replace('\n','', regex=True)
movies_27k_info = movies_27k_info.replace('  ','', regex=True)
movies_27k_info.to_csv('27k_data.csv',index=False)
movies_27k_info.to_pickle("27k_data.pkl")

https://www.rottentomatoes.com/m/hillsong_let_hope_rise
20000
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_disappointments_room
20001
https://www.rottentomatoes.com/m/the_magnificent_seven_2016
20002
https://www.rottentomatoes.com/m/snowden
20003
https://www.rottentomatoes.com/m/when_the_bough_breaks_2016
20004
https://www.rottentomatoes.com/m/storks
20005
https://www.rottentomatoes.com/m/sully
20006
https://www.rottentomatoes.com/m/hitchcocktruffaut
20007
https://www.rottentomatoes.com/m/greater
20008
something wrong with critic_consensus
https://www.rottentomatoes.com/m/denial_2016
20009
https://www.rottentomatoes.com/m/middle_school_the_worst_years_of_my_life
20010
something wrong with critic_consensus
https://www.rottentomatoes.com/m/31
20011
https://www.rottentomatoes.com/m/operation_avalanche
20012
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_monkey_king_2
20013
something wrong with critic_consensus
https://www.rottentom

something wrong with critic_consensus
https://www.rottentomatoes.com/m/nerve_2016
20109
https://www.rottentomatoes.com/m/papa_hemingway_in_cuba
20110
https://www.rottentomatoes.com/m/we_the_people_the_market_basket_effect
20111
something wrong with critic_consensus
https://www.rottentomatoes.com/m/fear_inc
20112
something wrong with critic_consensus
https://www.rottentomatoes.com/m/jack_goes_home
20113
something wrong with critic_consensus
https://www.rottentomatoes.com/m/mr_church
20114
something wrong with critic_consensus
https://www.rottentomatoes.com/m/front_cover_2016
20115
something wrong with critic_consensus
https://www.rottentomatoes.com/m/spaceman_2016
20116
something wrong with critic_consensus
https://www.rottentomatoes.com/m/autumn_lights
20117
something wrong with critic_consensus
https://www.rottentomatoes.com/m/into_the_inferno
20118
https://www.rottentomatoes.com/m/batman_return_of_the_caped_crusaders
20119
https://www.rottentomatoes.com/m/the_windmill_2016
20120
some

something wrong with critic_consensus
https://www.rottentomatoes.com/m/ace_the_case_manhattan_mystery_2016
20216
something wrong with critic_consensus
https://www.rottentomatoes.com/m/floyd_norman_an_animated_life
20217
something wrong with critic_consensus
https://www.rottentomatoes.com/m/kampai_for_the_love_of_sake
20218
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_story_of_the_last_chrysanthemums
20219
something wrong with critic_consensus
https://www.rottentomatoes.com/m/now_you_see_me_2
20220
https://www.rottentomatoes.com/m/the_conjuring_2
20221
https://www.rottentomatoes.com/m/captain_america_civil_war
20222
https://www.rottentomatoes.com/m/misconception
20223
something wrong with critic_consensus
https://www.rottentomatoes.com/m/hockney
20224
https://www.rottentomatoes.com/m/equals_2016
20225
https://www.rottentomatoes.com/m/take_me_to_the_river_2016
20226
something wrong with critic_consensus
https://www.rottentomatoes.com/m/popstar_never_stop_nev

https://www.rottentomatoes.com/m/i_saw_the_light
20323
https://www.rottentomatoes.com/m/my_golden_days
20324
https://www.rottentomatoes.com/m/600_miles
20325
something wrong with critic_consensus
https://www.rottentomatoes.com/m/a_sinner_in_mecca
20326
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_adderall_diaries
20327
https://www.rottentomatoes.com/m/the_debt_2016
20328
something wrong with critic_consensus
https://www.rottentomatoes.com/m/my_big_night_2016
20329
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_family_fang
20330
https://www.rottentomatoes.com/m/no_men_beyond_this_point
20331
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_dresser
20332
https://www.rottentomatoes.com/m/mountains_may_depart_2016
20333
https://www.rottentomatoes.com/m/outlaws_and_angels
20334
https://www.rottentomatoes.com/m/mei_ren_yu
20335
https://www.rottentomatoes.com/m/brahman_naman
20336
something wrong with critic_c

something wrong with critic_consensus
https://www.rottentomatoes.com/m/kindergarten_cop_2
20431
something wrong with critic_consensus
https://www.rottentomatoes.com/m/manhattan_night
20432
something wrong with critic_consensus
https://www.rottentomatoes.com/m/memoria_2016
20433
something wrong with critic_consensus
https://www.rottentomatoes.com/m/urge
20434
something wrong with critic_consensus
https://www.rottentomatoes.com/m/hard_sell
20435
something wrong with critic_consensus
https://www.rottentomatoes.com/m/bo_burnham_make_happy
20436
something wrong with critic_consensus
https://www.rottentomatoes.com/m/pele_birth_of_a_legend_2016
20437
something wrong with critic_consensus
https://www.rottentomatoes.com/m/east_side_sushi
20438
something wrong with critic_consensus
https://www.rottentomatoes.com/m/regression
20439
https://www.rottentomatoes.com/m/ride_along_2
20440
https://www.rottentomatoes.com/m/joy_2014
20441
https://www.rottentomatoes.com/m/the_fifth_wave
20442
https://www.r

...

https://www.rottentomatoes.com/m/in_between_days
26289
https://www.rottentomatoes.com/m/hairspray
26290
https://www.rottentomatoes.com/m/live_free_or_die_hard
26291
https://www.rottentomatoes.com/m/this_is_england
26292
https://www.rottentomatoes.com/m/rescue_dawn
26293
https://www.rottentomatoes.com/m/la_vie_en_rose
26294
https://www.rottentomatoes.com/m/skinwalkers
26295
https://www.rottentomatoes.com/m/provoked_a_true_story
26296
https://www.rottentomatoes.com/m/cut_sleeve_boys
26297
something wrong with critic_consensus
https://www.rottentomatoes.com/m/10008005-moliere
26298
https://www.rottentomatoes.com/m/your_mommy_kills_animals
26299
https://www.rottentomatoes.com/m/the_heartbreak_kid
26300
https://www.rottentomatoes.com/m/helvetica
26301
something wrong with critic_consensus
https://www.rottentomatoes.com/m/chak_de_india
26302
something wrong with critic_consensus
https://www.rottentomatoes.com/m/sawdust_and_tinsel
26303
something wrong with critic_consensus
https://www.rotten

https://www.rottentomatoes.com/m/severance
26402
https://www.rottentomatoes.com/m/laura_smiles
26403
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_valet
26404
https://www.rottentomatoes.com/m/we_are_marshall
26405
https://www.rottentomatoes.com/m/election_2
26406
https://www.rottentomatoes.com/m/insiang_2007
26407
something wrong with critic_consensus
https://www.rottentomatoes.com/m/flying_scotsman
26408
https://www.rottentomatoes.com/m/lucky_you
26409
https://www.rottentomatoes.com/m/scarecrows
26410
something wrong with critic_consensus
https://www.rottentomatoes.com/m/two_weeks
26411
https://www.rottentomatoes.com/m/every_again
26412
something wrong with critic_consensus
https://www.rottentomatoes.com/m/robinson_crusoe_on_mars
26413
something wrong with critic_consensus
https://www.rottentomatoes.com/m/i_have_never_forgotten_you_the_life_and_legacy_of_simon_wiesenthal
26414
something wrong with critic_consensus
https://www.rottentomatoes.com/m/even_mone

something wrong with critic_consensus
https://www.rottentomatoes.com/m/factory_girl
26521
https://www.rottentomatoes.com/m/race-you-to-the-bottom
26522
something wrong with critic_consensus
https://www.rottentomatoes.com/m/shooter
26523
https://www.rottentomatoes.com/m/black_snake_moan
26524
https://www.rottentomatoes.com/m/hills_have_eyes_2
26525
https://www.rottentomatoes.com/m/mimzy
26526
https://www.rottentomatoes.com/m/dead_silence
26527
https://www.rottentomatoes.com/m/10005819-10005819-neverwas
26528
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the_giant_behemoth
26529
something wrong with critic_consensus
https://www.rottentomatoes.com/m/ta_ra_rum_pum
26530
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1178348-the_last_time
26531
something wrong with critic_consensus
https://www.rottentomatoes.com/m/something_to_cheer_about
26532
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1001323-attack_of_the_50_foo

https://www.rottentomatoes.com/m/ball_of_fire
26628
https://www.rottentomatoes.com/m/american-pastime
26629
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1003119-broken_arrow
26630
something wrong with critic_consensus
https://www.rottentomatoes.com/m/prince_of_the_city
26631
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1175858-simple_curve
26632
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1003436-camelot
26633
something wrong with critic_consensus
https://www.rottentomatoes.com/m/harry_and_the_hendersons
26634
something wrong with critic_consensus
https://www.rottentomatoes.com/m/alice_sweet_alice
26635
something wrong with critic_consensus
https://www.rottentomatoes.com/m/shirley_valentine
26636
something wrong with critic_consensus
https://www.rottentomatoes.com/m/iraq_in_fragments
26637
https://www.rottentomatoes.com/m/1012225-les_miserables
26638
something wrong with critic_consensus
https://www.rottento

something wrong with critic_consensus
https://www.rottentomatoes.com/m/dukes_of_hazzard_the_beginning
26741
something wrong with critic_consensus
https://www.rottentomatoes.com/m/manitou
26742
something wrong with critic_consensus
https://www.rottentomatoes.com/m/sublime
26743
something wrong with critic_consensus
https://www.rottentomatoes.com/m/muriel-1963
26744
something wrong with critic_consensus
https://www.rottentomatoes.com/m/return_of_frank_james
26745
something wrong with critic_consensus
https://www.rottentomatoes.com/m/1002407-billy_budd
26746
something wrong with critic_consensus
https://www.rottentomatoes.com/m/confessions_of_a_burning_man
26747
something wrong with critic_consensus
https://www.rottentomatoes.com/m/fall_to_grace
26748
something wrong with critic_consensus
https://www.rottentomatoes.com/m/lovewrecked
26749
something wrong with critic_consensus
https://www.rottentomatoes.com/m/the-heart-of-the-game
26750
https://www.rottentomatoes.com/m/crossover
26751
http

In [ ]:
# page = requests.get("https://www.rottentomatoes.com/m/bobby_fischer_against_the_world")
# soup=bs(page.text)
# movie_name = soup.find('div',{'class':'mop-ratings-wrap score_panel js-mop-ratings-wrap'})
# movie_name = soup.findAll('h1',{'class':'mop-ratings-wrap__title mop-ratings-wrap__title--top'})[0].get_text()


In [68]:
# movies_1400_data = scrape_rotten(movie_url_genre)
movies_1400_data = movies_1400_data.replace('\n','', regex=True)
movies_1400_data = movies_1400_data.replace('  ','', regex=True)
movies_1400_data.to_csv("movies_1400_data.csv", index=False)
movies_1400_data.to_pickle("movies_1400_data.pkl")

### Movie name and consensus

In [225]:
page = requests.get("https://www.rottentomatoes.com/m/seven_samurai_1956")
soup=bs(page.text)


movie_name = soup.find('div',{'class':'mop-ratings-wrap score_panel js-mop-ratings-wrap'})
movie_name = soup.findAll('h1',{'class':'mop-ratings-wrap__title mop-ratings-wrap__title--top'})[0].get_text()
critic_consensus = soup.findAll('p',{'class':'mop-ratings-wrap__text mop-ratings-wrap__text--concensus'})[0].get_text()

In [226]:
movie_name

'Seven Samurai (Shichinin no Samurai)'

In [228]:
critic_consensus

"Arguably Akira Kurosawa's masterpiece, The Seven Samurai is an epic adventure classic with an engrossing story, memorable characters, and stunning action sequences that make it one of the most influential films ever made."

## Ratings

In [209]:
critic = soup.find('div',{'class':'mop-ratings-wrap__half'})

In [107]:
try:
    critic_score = critic.find('span',{'class':'mop-ratings-wrap__percentage'}).get_text()
    print(critic_score)
except AttributeError:
    critic_score = np.NaN


                    100%
                


In [108]:
num_critics = critic.find('small',{'class':'mop-ratings-wrap__text--small'}).get_text()
num_critics.replace('\n','')

'                            64                    '

In [109]:
audience = soup.find('div',{'class':'mop-ratings-wrap__half audience-score'})
try:
    audience_score = audience.find('span',{'class':'mop-ratings-wrap__percentage'}).get_text()
    print(audience_score)
except AttributeError:
    audience_score = np.NaN


                    97%
                


In [110]:
num_audience = audience.find('strong',{'class':'mop-ratings-wrap__text--small'}).get_text()
num_audience

'User Ratings: 90,992'

### Movie Info

In [163]:
tables = soup.find('div',{'class':'panel-body content_body'})
synopsis = tables.findAll('div',{'class':'movie_synopsis clamp clamp-6 js-clamp'})[0].get_text()
synopsis.replace('\n','')

"                Akira Kurosawa's epic tale concerns honor and duty during a time when the old traditional order is breaking down. The film opens with master samurai Kambei (Takashi Shimura) posing as a monk to save a kidnapped farmer's child. Impressed by his selflessness and bravery, a group of farmers begs him to defend their terrorized village from bandits. Kambei agrees, although there is no material gain or honor to be had in the endeavor. Soon he attracts a pair of followers: a young samurai named Katsushiro (Isao Kimura), who quickly becomes Kambei's disciple, and boisterous Kikuchiyo (Toshiro Mifune), who poses as a samurai but is later revealed to be the son of a farmer. Kambei assembles four other samurais, including Kyuzo (Seiji Miyaguchi), a master swordsman, to round out the group. Together they consolidate the village's defenses and shape the villagers into a militia, while the bandits loom menacingly nearby. Soon raids and counter-raids build to a final bloody heart-wre

In [162]:
pg = tables.find('div',{'class':'meta-value'}).get_text()
pg.replace('\n','')

'G                    '

In [173]:
synopsis = tables.findAll('li',{'class':'meta-row clearfix'})
genre = synopsis[1].find('div',{'class':'meta-value'}).get_text()
genre.replace('\n','')

'                                                Action & Adventure,                                                 Art House & International,                                                 Classics,                                                 Drama                                            '

In [114]:
directors = synopsis[2].findAll('div',{'class':'meta-value'})[0].get_text()
directors.replace('\n','')

'Akira Kurosawa'

In [115]:
written_by = synopsis[3].findAll('div',{'class':'meta-value'})[0].get_text()
written_by.replace('\n','')

'Akira Kurosawa,                                                 Shinobu Hashimoto,                                                 Hideo Oguni'

In [116]:
release_date = synopsis[4].findAll('div',{'class':'meta-value'})[0].get_text()
release_date.replace('\n','')

'Nov 19, 1956                        \xa0wide'

In [120]:
run_time = synopsis[7].findAll('div',{'class':'meta-value'})[0].get_text()
run_time

'\n                        Columbia Pictures\n                    '

dict_data = {}

for i in range(12):
    try:
        ii = synopsis[i].findAll('div',{'class':'meta-label subtle'})[0].get_text()
        dict_data[ii] = synopsis[i].find('div',{'class':'meta-value'}).get_text()
    except:
        continue

In [119]:
try:
    studio = synopsis[8].findAll('div',{'class':'meta-value'})[0].get_text()

except IndexError:
    studio = np.NaN
print(studio)    


nan


## Cast

In [ ]:
def list_celeb_names(all_celebrities):
    top_5_celeb_names = []
    try:
        for i in range(5):
            top_5_celeb_names.append(all_celebrities[i].attrs['href'][11:])
    except IndexError:
        continue
    return top_5_celeb_names

In [ ]:
cast = soup.find('div',{'class':'castSection'})
try :
    all_celebs = cast.findAll('a',{'class':'pull-left'})
    top_5_celeb_names = list_celeb_names(all_celebs)
#     print(top_5_celeb_names)
except AttributeError:
    pass
top_5_celeb_names = ", ".join(top_5_celeb_names)
top_5_celeb_names

## Critics Review

In [ ]:
def critic_line_url(critic_lines):
    url =[]
    critic_line =[]
    for i in range(20):
        try:
            url.append(critic_lines[i].attrs['cite'])
            critic_line.append(critic_lines[i].get_text())
        except:
            continue
    return critic_line, url

In [ ]:
# critic_review = soup.find('section',{'class':'panel panel-rt panel-box'})
critic_lines = soup.findAll('div',{'class':'panel-body content_body'})[3]
critic_lines = critic_lines.findAll('blockquote',{'class':'media-body quote_bubble__quote'})
critic_line, critic_url = critic_line_url(critic_lines)
critic_line, critic_url = ", ".join(critic_line), ", ".join(critic_url)
critic_url

## Audiences Review

In [ ]:
def audience_line(audience_review):
    audience_line =[]
    for i in range(10):
        try:
            audience_line.append(audience_review.findAll('div',{'class':'mop-audience-reviews__review--comment clamp clamp-4 js-clamp'})[i].get_text())
        except IndexError:
            continue
    return audience_line

In [ ]:
audience_review = soup.find('ul',{'class':'mop-audience-reviews__reviews-wrap clearfix'})
audience_lines = audience_line(audience_review)
audience_review = "".join(audience_lines)
audience_review

## Quotes

In [ ]:
quotes_url = soup.findAll('div',{'class':'panel-body content_body'})[4]
quotes_url = quotes_url.find('a',{'class':'unstyled articleLink fr'}).attrs['href']
quotes_url = 'https://www.rottentomatoes.com' + quotes_url
quotes_page = requests.get(quotes_url)
quotes_page=bs(quotes_page.text)

In [ ]:
quotes_lines = quotes_page.findAll('div',{'class':'panel-body content_body'})[1]
# quotes_lines.findAll('span',{'class':'bold quote_actor'})
quotes_lines = quotes_lines.findAll('span')

In [ ]:
quote_lines = quotes_lines[1::2]
# [word for word in re.split("\W+",quote_lines) if word.lower() not in ['<span>']]
cleaned_quotes = []
for line in quote_lines:
    try:
        line = str(line).replace('\n','')
        cleaned_quotes.append(re.search('<span>(.*)</span>',line).group(1))
    except:
        continue

quotes = "".join(cleaned_quotes)
# quotes

In [ ]:
# for i in list(quotes_lines):
#     items = re.findall("bold quote_actor.*$",str(i),re.MULTILINE)
#     print(items)

In [ ]:
# ,'num_critics','audience_score','num_audience',
#                           'synopsis','pg','genre','directors','written_by','release_date',
#                            'run_time','studio','top_5_celeb_names', 'critic_line', 'critic_url',
#                             'audience_review', 'quotes'

In [ ]:
df=pd.DataFrame()
df['critic_score'] = [critic_score]
df['audience_score'] = [audience_score]
df['pg'] = pg
df['genre'] = genre
df['directors'] = directors
df['written_by'] = written_by
df['release_date'] = release_date
df['run_time'] = run_time
df['studio'] = studio
df['top_5_celeb_names'] = top_5_celeb_names
df['critic_line'] = critic_line
df['critic_url'] = critic_url
df['audience_review'] = audience_review
df['quotes'] = quotes

In [ ]:
df